**Authors:** Jozef Hanč, Martina Hančová <br> *[Faculty of Science](https://www.upjs.sk/en/faculty-of-science/?prefferedLang=EN), P. J. Šafárik University in Košice, Slovakia* <br> emails: [jozef.hanc@upjs.sk](mailto:jozef.hanc@upjs.sk)
***

# <font color = brown, size=6> Measuring runtime in MATLAB </font>

<font size=5> Computational tools: </font>  **<font size=5>MATLAB</font>** 

code `time_it.m` 

---




### Example - Characteristic function of $X_1$
$\varphi_{X_1}(t_1)$

Random variable $ X_1 \sim N(a, 1), a = 1.5 $:
- CF: $ \varphi_{X_1}(t) = e^{i a t - \frac{t^2}{2}}$

In [1]:
% characteristic function of X1 ~ N(a,1)
a = 1.5;
cf1 = @(t1) exp(1i * a * t1 - t1.^2 / 2);

# Measuring runtime

To measure runtime in MATLAB, you can use two primary built-in methods: `timeit` and `tic/toc`.

### Matlab built-in `tic/toc`
- `tic/toc`: Use for quick, single timing of code blocks. It's less precise due to potential system noise and runtime variability but convenient for exploratory timing.

In [2]:
tic;
t1 = linspace(-4.5, 7, 10000);
cft = cf1(t1);
runtime = toc

runtime = 0.0027

### Matlab built-in function `timeit`
- `timeit`: Use for precise benchmarking, as it averages multiple timings, ensuring reliability by reducing variability and excluding setup overhead. MATLAB doesn't provide an exact number of iterations for each timing because it adapts dynamically based on the function's characteristics. 

In [3]:
t1 = linspace(-4.5, 7, 10000);
runtime = timeit(@() cf1(t1));
runtime

runtime = 5.4809e-05

### Our custom Matlab function `time_it`
- `time_it.m`: Combines the flexibility of timing both functions and inline code snippets. Allows control over repetitions and loops, giving detailed statistics (average, best, worst, standard deviation, CV).

In [4]:
% Example Usage
% Define input
t1_values = linspace(-4.5, 7, 10000);

% Set the options for time_it
options.r = 10000; % Set the number of runs
options.n = 3;  % Set the number of loops  per run
options.func_args = {t1_values}; % Arguments to be passed to the function

### measuring runtime of a function

In [5]:
% Measure the runtime of the function using time_it with options
runtime = time_it(cf1, options);

% Access and display the timing results
disp(runtime.output)

5.15e-05 s ± 8.95e-06 s (17% of mean) per loop (mean ± std. dev. of 10000 runs, 3 loops each)


In [6]:
% complete results
disp(runtime)

     output: '5.15e-05 s ± 8.95e-06 s (17% of mean) per loop (mean ± std. dev. of 10000 runs, 3 loops each)'
    allruns: [0.0015 0.0012 1.8670e-04 1.8000e-04 2.7430e-04 2.1660e-04 1.5120e-04 1.5540e-04 1.4920e-04 1.5890e-04 1.5000e-04 1.5020e-04 1.5260e-04 2.0710e-04 ... ] (1x10000 double)
    average: 5.1516e-05
       best: 3.9833e-05
      worst: 5.0740e-04
      stdev: 8.9482e-06
         cv: '17.37%'



### measuring runtime of a code snippet

In [7]:
% Define input values
a = 1.5;
cf1 = @(t1) exp(1i * a * t1 - t1.^2 / 2);
t1_values = linspace(-4.5, 7, 10000);

In [8]:
code = 'a = 1.5; cf1 = @(t1) exp(1i * a * t1 - t1.^2 / 2); t1_values = linspace(-4.5, 7, 10000);';

In [9]:
% Code snippet to measure (as a string)
code = 'a = 1.5; cf1 = @(t1) exp(1i * a * t1 - t1.^2 / 2); t1_values = linspace(-4.5, 7, 10000);';

% Define options for timing
options = struct('r', 10000, 'n', 3); % Adjust repetitions and loops as needed

% Measure runtime using the updated time_it function
runtime = time_it(code, options);

% Display the runtime output
disp(runtime.output);

2.70e-05 s ± 1.09e-05 s (40% of mean) per loop (mean ± std. dev. of 10000 runs, 3 loops each)


In [10]:
disp(runtime)

     output: '2.70e-05 s ± 1.09e-05 s (40% of mean) per loop (mean ± std. dev. of 10000 runs, 3 loops each)'
    allruns: [0.0019 0.0017 1.1380e-04 9.4700e-05 1.6170e-04 9.6200e-05 7.4100e-05 9.6600e-05 7.4500e-05 7.6200e-05 7.4000e-05 7.5300e-05 7.6400e-05 6.7400e-05 ... ] (1x10000 double)
    average: 2.7003e-05
       best: 2.1033e-05
      worst: 6.3517e-04
      stdev: 1.0894e-05
         cv: '40.34%'



## MATLAB code `time_it.m`

In [11]:
function runtime = time_it(code_input, options)
    % time_it - Measure the runtime of a function or code snippet.
    % The function is the simple equivalent of Python's %timeit.
    %
    % Usage:
    % runtime = time_it(code_input, options)
    %
    % Input:
    % - code_input: A function handle for the function to be measured or a string with code to be executed.
    % - options: A structure with optional fields:
    %   - r: Number of repetitions (default = 3)
    %   - n: Number of loops per repetition (default = 1000)
    %   - func_args: Cell array of arguments to be passed to the function (default = {})
    %
    % Output:
    % - runtime: A structure containing the timing statistics including allruns,
    %   average, best, worst, standard deviation (stdev), and coefficient of variation (cv_percent).
    %
    % 2024 (c) Hanc, Hancova (jozef.hanc@gmail.com)
    % Ver.: 11-November-2024 

    % Check if at least one argument is provided
    if nargin < 1
        error('time_it:InsufficientInput', 'The code input must be provided as the argument.');
    end

    % Set default values for options
    if nargin < 2, options = struct(); end

    if ~isfield(options, 'r')
        options.r = 3; % Default number of repetitions
    end

    if ~isfield(options, 'n')
        options.n = 1000; % Default number of loops per repetition
    end

    if ~isfield(options, 'func_args')
        options.func_args = {}; % Default arguments for the function
    end

    % Extract options
    r = options.r;
    n = options.n;
    func_args = options.func_args;

    % Preallocate array to store the time taken for each repetition
    allruns = zeros(1, r);

    % Determine if the input is a function handle or a string of code
    if isa(code_input, 'function_handle')
        % Execute the provided function handle r times, each time executing it n times
        for i = 1:r
            tic;
            for j = 1:n
                code_input(func_args{:}); % Call the function handle with its arguments (if any)
            end
            allruns(i) = toc;
        end
    elseif ischar(code_input) || isstring(code_input)
        % Execute the provided code snippet r times, each time executing it n times
        for i = 1:r
            tic;
            for j = 1:n
                evalin('base', code_input); % Evaluate the code snippet in the base workspace
            end
            allruns(i) = toc;
        end
    else
        error('time_it:InvalidInput', 'The code input must be either a function handle or a string of code.');
    end

    % Calculate per-loop times
    per_loop_times = allruns / n;

    % Calculate summary characteristics
    mean_time = mean(per_loop_times);
    std_time = std(per_loop_times);
    best_time = min(per_loop_times);
    worst_time = max(per_loop_times);

    % the coefficient of variation (cv) in %
    if mean_time ~= 0
        cv = (std_time / mean_time) * 100;
    else
        cv = NaN; % Handle division by zero if the average is zero
    end

    % Store timing results in a structure based on per-loop times
    runtime.output = sprintf('%.2e s ± %.2e s (%.2g%% of mean) per loop (mean ± std. dev. of %d runs, %d loops each)', mean_time, std_time, cv, r, n);
    runtime.allruns = allruns;
    runtime.average = mean_time;
    runtime.best = best_time;
    runtime.worst = worst_time;
    runtime.stdev = std_time;
    runtime.cv = sprintf('%.2f%%', cv);
end

% Original code for executing strings without 'evalin':
% elseif ischar(code_input) || isstring(code_input)
%     % Execute the provided code snippet r times, each time executing it n times
%     for i = 1:r
%         tic;
%         for j = 1:n
%             eval(code_input); % Evaluate the code snippet
%         end
%         allruns(i) = toc;
%     end
